In [70]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

### Acquire the train.csv

In [2]:
# Loade the csv files
train = pd.read_csv("train.csv")

# Inspect the shapes
train.shape

(101230332, 10)

In [3]:
# Take a quck peek of the dataframe
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [4]:
# Summarize the train dataset
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 7.5+ GB


**Takeaways**
- The train dataset has more than 100 millions records. I only need to small portion to calculate the statistics of the past.<br>
    -**I will take the first 1000 user ids.**

In [8]:
# Extract the first 1000 user ids

user_ids1000 = train.user_id.value_counts().sort_index().iloc[0: 1000]
user_ids1000 = user_ids1000.index.to_list()

# Sanity check
print(len(user_ids1000))

# Print the first 5 user ids
user_ids1000[:5]

1000


[115, 124, 2746, 5382, 8623]

In [10]:
# Set the user_id to the index

train = train.set_index("user_id")
train.head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,NaN,NaN
115,1,56943,5716,0,2,2,1,37000.0,False
115,2,118363,128,0,0,0,1,55000.0,False
115,3,131167,7860,0,3,0,1,19000.0,False
115,4,137965,7922,0,4,1,1,11000.0,False


In [11]:
# Subtracts the records for the first 1000 users
train1000 = train.loc[user_ids1000]

# Take a peek at the new dataset
train1000.head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,NaN,NaN
115,1,56943,5716,0,2,2,1,37000.0,False
115,2,118363,128,0,0,0,1,55000.0,False
115,3,131167,7860,0,3,0,1,19000.0,False
115,4,137965,7922,0,4,1,1,11000.0,False


In [12]:
# Sanity check
train1000.tail() # should match the tail of user_ids1000

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
5577726,296610,625311764,5328,0,7,0,0,20000.0,False
5577726,296611,625327721,4303,0,8,3,0,10000.0,True
5577726,296612,625344560,4445,0,9,0,1,6000.0,True
5577726,296613,625367161,5206,0,10,1,0,6000.0,True
5577726,296614,625378759,3928,0,11,0,0,13000.0,True


In [14]:
# Print the tail of the user_ids1000
user_ids1000[999]

5577726

In [60]:
# Read the train1000.csv
train1000 = pd.read_csv("train1000.csv", index_col=0)

In [61]:
# Quick summarize the train1000
train1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296615 entries, 115 to 5577726
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   row_id                          296615 non-null  int64  
 1   timestamp                       296615 non-null  int64  
 2   content_id                      296615 non-null  int64  
 3   content_type_id                 296615 non-null  int64  
 4   task_container_id               296615 non-null  int64  
 5   user_answer                     296615 non-null  int64  
 6   answered_correctly              296615 non-null  int64  
 7   prior_question_elapsed_time     289945 non-null  float64
 8   prior_question_had_explanation  295617 non-null  object 
dtypes: float64(1), int64(7), object(1)
memory usage: 22.6+ MB


In [62]:
# Count how many missing values in the prior_question_elapsed_time

mask = train1000.prior_question_elapsed_time.isnull()
mask.sum()

6670

In [63]:
# Count how many missing values in the prior_question_had_explanation

mask = train1000.prior_question_had_explanation.isnull()
mask.sum()

998

**Takeaways**
- The memoery usage of the first 1000 users are only 23 MB
- Missing values are found in columns:
    - prior_question_elapsed_time: 6670 nulls
    - prior_question_had_explanantion: 998 nulls

### Handling the null values

In [64]:
# The values of the two prior columns will always be 0 when the user first interacted with the system
# Take a first user 115 for example

train1000.loc[115].head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,NaN,NaN
115,1,56943,5716,0,2,2,1,37000.0,False
115,2,118363,128,0,0,0,1,55000.0,False
115,3,131167,7860,0,3,0,1,19000.0,False
115,4,137965,7922,0,4,1,1,11000.0,False


**Takeawasy**
- There are only 1000 users so the two prior columns should have at leat 1000 nulls.
- The prior_question_had_explanantion column has 998 nulls:
    - fill the nulls with 'False'
- I will further explore the missing values in the column prior_question_elapased_time.

### Fill the nulls in the prior_question_had_explanation with string 'False'

In [65]:
train1000.prior_question_had_explanation.fillna('False', inplace = True)
train1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296615 entries, 115 to 5577726
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   row_id                          296615 non-null  int64  
 1   timestamp                       296615 non-null  int64  
 2   content_id                      296615 non-null  int64  
 3   content_type_id                 296615 non-null  int64  
 4   task_container_id               296615 non-null  int64  
 5   user_answer                     296615 non-null  int64  
 6   answered_correctly              296615 non-null  int64  
 7   prior_question_elapsed_time     289945 non-null  float64
 8   prior_question_had_explanation  296615 non-null  object 
dtypes: float64(1), int64(7), object(1)
memory usage: 32.6+ MB


In [66]:
mask = train1000.prior_question_elapsed_time.isnull()
train1000[mask]

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,NaN,False
124,46,0,7900,0,0,0,1,NaN,False
2746,76,0,5273,0,0,1,0,NaN,False
2746,89,653762,6808,1,14,-1,-1,NaN,False
5382,96,0,5000,0,0,0,1,NaN,False
...,...,...,...,...,...,...,...,...,...
5566848,296498,162115714,32168,1,21,-1,-1,NaN,False
5568049,296509,0,6028,0,0,3,0,NaN,False
5568049,296537,7746409318,24250,1,20,-1,-1,NaN,False


**Takeaways**
- When the user studies the lecture, the prior_question_elapsed_time documented as NaN.
- I will fill the missing values in the columsn prior_question_elapsed_time with 0.

### Fill the nulls in the prior_question_elapsed_time with string 0

In [67]:
train1000.prior_question_elapsed_time.fillna(0, inplace = True)
train1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296615 entries, 115 to 5577726
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   row_id                          296615 non-null  int64  
 1   timestamp                       296615 non-null  int64  
 2   content_id                      296615 non-null  int64  
 3   content_type_id                 296615 non-null  int64  
 4   task_container_id               296615 non-null  int64  
 5   user_answer                     296615 non-null  int64  
 6   answered_correctly              296615 non-null  int64  
 7   prior_question_elapsed_time     296615 non-null  float64
 8   prior_question_had_explanation  296615 non-null  object 
dtypes: float64(1), int64(7), object(1)
memory usage: 32.6+ MB


**Takeawasy**
- All the null values have been addressed in the dataset. 

### Split the train1000 into train and test

In [69]:
# Double check the 1000 user_id list
print(len(user_ids1000))
user_ids1000[:5]

1000


[115, 124, 2746, 5382, 8623]

In [121]:
# Create the train and test dataframe

train = pd.DataFrame()
test = pd.DataFrame()

# Set up the train size
train_size = 0.8

for user_id in user_ids1000:
    if train1000.loc[[user_id]].shape[0] <=2:
        print(user_id)
        continue
    elif train1000.loc[[user_id]].shape[0] > 2: 
        df = train1000.loc[[user_id]]
        n = df.shape[0]
        test_start_index = round(train_size * n)
        df_train = df.iloc[:test_start_index]
        df_test = df.iloc[test_start_index:]     
        train = pd.concat([train, df_train])
        test = pd.concat([test, df_test])
    
# Print the shape of the original, train and test
train1000.shape, train.shape, test.shape

1880240
2148001


((296615, 9), (237322, 9), (59290, 9))

In [122]:
# Take a peek at the train dataset

train.head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,0.0,False
115,1,56943,5716,0,2,2,1,37000.0,False
115,2,118363,128,0,0,0,1,55000.0,False
115,3,131167,7860,0,3,0,1,19000.0,False
115,4,137965,7922,0,4,1,1,11000.0,False


In [123]:
# Take a peek at the test

test.head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,37,1415188,7882,0,37,2,1,20000.0,True
115,38,1468285,7962,0,38,3,1,17000.0,True
115,39,667861680,1278,0,39,3,1,21000.0,False
115,40,667971812,2065,0,40,2,1,17000.0,False
115,41,667971812,2064,0,40,1,1,17000.0,False


### Count how many lectures each id has reviewed

In [57]:
# Create a new columns for if the student review the lecture or not

train1000['reviewed_lecture'] = (train1000['answered_correctly'] == -1)
train1000

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,reviewed_lecture
user_id,,,,,,,,,,
115,0,0,5692,0,1,3,1,0.0,False,False
115,1,56943,5716,0,2,2,1,37000.0,False,False
115,2,118363,128,0,0,0,1,55000.0,False,False
115,3,131167,7860,0,3,0,1,19000.0,False,False
115,4,137965,7922,0,4,1,1,11000.0,False,False
...,...,...,...,...,...,...,...,...,...,...
5577726,296610,625311764,5328,0,7,0,0,20000.0,False,False
5577726,296611,625327721,4303,0,8,3,0,10000.0,True,False
5577726,296612,625344560,4445,0,9,0,1,6000.0,True,False


In [ ]:
# Compute how many times each user reviewed the lecture

train1000

### Feature engineering - how to measure the whole content

In [54]:
train1000.groupby('content_id').answered_correctly.agg(['count', 'mean', 'std', 'median', 'skew'])

,count,mean,std,median,skew
content_id,,,,,
0,23,0.956522,0.208514,1.0,-4.795832
1,16,0.937500,0.250000,1.0,-4.000000
2,134,0.574627,0.496255,1.0,-0.305317
3,66,0.787879,0.411943,1.0,-1.441340
4,88,0.522727,0.502346,1.0,-0.092589
...,...,...,...,...,...
32491,9,-1.000000,0.000000,-1.0,0.000000
32535,13,-1.000000,0.000000,-1.0,0.000000
32570,11,-1.000000,0.000000,-1.0,0.000000
